# Trabajo Integrador: DuckDB

## Instalacion

In [1]:
%pip install duckdb --upgrade


   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.3 MB 2.9 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.3 MB 2.2 MB/s eta 0:00:05
   ----- ---------------------------------- 1.6/11.3 MB 2.1 MB/s eta 0:00:05
   ------ --------------------------------- 1.8/11.3 MB 2.1 MB/s eta 0:00:05
   ------- -------------------------------- 2.1/11.3 MB 2.1 MB/s eta 0:00:05
   --------- ------------------------------ 2.6/11.3 MB 1.9 MB/s eta 0:00:05
   ----------- ---------------------------- 3.1/11.3 MB 2.0 MB/s eta 0:00:05
   ------------ --------------------------- 3.7/11.3 MB 2.0 MB/s eta 0:00:04
   --------------- ------------------------ 4.5/11.3 MB 2.3 MB/s eta 0:00:04
   ---------------- ----------------------- 4.7/11.3 MB 2.3 MB/s eta 0:00:03
   ------------------ --------------------- 5.2/11.3 MB 2.1 MB/s eta 0:00:03
   ---------


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Creando conexión

Nos permite establecer una conexión a una base de datos, por defecto, si no especificamos su nombre, la base de datos no persistirá y operará en memoria, por lo tanto no se almacenarán las tablas creadas.

In [2]:
import duckdb

BD = duckdb.connect("main.db")

Prueba de conexión

In [3]:
BD.read_csv("dataset/olist_customers_dataset.csv")

BD.sql("SELECT * FROM 'dataset/olist_customers_dataset.csv' LIMIT 10")

┌──────────────────────────────────┬──────────────────────────────────┬──────────────────────────┬───────────────────────┬────────────────┐
│           customer_id            │        customer_unique_id        │ customer_zip_code_prefix │     customer_city     │ customer_state │
│             varchar              │             varchar              │         varchar          │        varchar        │    varchar     │
├──────────────────────────────────┼──────────────────────────────────┼──────────────────────────┼───────────────────────┼────────────────┤
│ 06b8999e2fba1a1fbc88172c00ba8bc7 │ 861eff4711a542e4b93843c6dd7febb0 │ 14409                    │ franca                │ SP             │
│ 18955e83d337fd6b2def6b18a428ac77 │ 290c77bc529b7ac935b93aa66c333dc3 │ 09790                    │ sao bernardo do campo │ SP             │
│ 4e7b3e00288586ebd08712fdd0374a03 │ 060e732b5b29e8181a18229c7b0b2b5e │ 01151                    │ sao paulo             │ SP             │
│ b2b6027bc5c5109e52